In [5]:
import enchant
import json
import glob
from lxml import etree
import os
from enchant.checker import SpellChecker
import csv

In [3]:
def lire_TEI_XML(input_file):
    namespace = "{http://www.tei-c.org/ns/1.0}text"
    parser = etree.XMLParser(recover=True)
    root = etree.parse(input_file, parser)
    contenu = ""
    for texte in root.iter(namespace):
        textes = texte.itertext()
        for cpt, el in enumerate(textes):
            if el != "\n":
                contenu += el
                contenu += " "
        contenu = contenu.replace("\n", " ")
    return contenu

In [16]:
dossier_corpus = "Corpus/test"

repertoire_correction = "%s_correction2" % dossier_corpus
namespace = "{http://www.tei-c.org/ns/1.0}"
if not os.path.exists(repertoire_correction):
    os.makedirs(repertoire_correction)
    
chkr = SpellChecker("fr")

with open("%s/erreurs.csv" % repertoire_correction, "w", encoding="utf-8", newline='') as csvfile:
    fieldnames = ["Erreur détectée", "Correction proposée", "Fichier"]
    spamwriter = csv.writer(csvfile, delimiter=",")
    spamwriter.writerow(fieldnames)

    for fichier in glob.glob("%s/*" % dossier_corpus):
        extension = fichier.split(".")[-1]
        if extension == "txt" or extension == "xml":
            nom_fichier = fichier.split("\\")[1]
            sans_extension = nom_fichier.split(".")[-2]
            print(sans_extension)
            parser = etree.XMLParser(recover=True)
            root = etree.parse(fichier, parser)
            if extension == "xml":
                contenu = lire_TEI_XML(fichier)
            else:
                with open(fichier, 'r', encoding = "utf-8") as fin:
                    contenu = fin.read()
            #print(len(contenu))
            chkr.set_text(contenu)
            for cpt, err in enumerate(chkr):
                ligne = []
                ligne.append(err.word)
                if len(chkr.suggest(err.word)) > 0:
                    err.replace(chkr.suggest(err.word)[0])
                    ligne.append(chkr.suggest(err.word)[0])
                else:
                    ligne.append("Pas de correction trouvée")
                #ligne.append(err.leading_context(50))
                #ligne.append(err.trailing_context(50))
                ligne.append(nom_fichier)
                spamwriter.writerow(ligne)
                if cpt > 10:
                    break
            #break
            correction_contenu = chkr.get_text()
            file = open("%s/%s.txt" % (repertoire_correction, sans_extension), "w", encoding="utf-8")
            file.write(correction_contenu)
            file.close()
    #break

5431000
5652000


In [13]:
import csv
with open('eggs.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|')
    spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
    spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])